# Load in data - Bitcoin Tweets 2014-2019

In [50]:
storage_account = "ds562team9datalake"
container_name = "bronze"
file_path = "Crypto_tweets/bitcoinTweets_2014_2019.csv"

storage_key = "KXg2Djg7uRevBSpPNIVnKw/N6HpqBh+kJwDX07wkywbpU2joMZdTIBOXk30EoMMxH2d8wwb+9j0g+AStO60IWw=="

spark.conf.set(f"fs.azure.account.key.{storage_account}.blob.core.windows.net", storage_key)

url = f"wasbs://{container_name}@{storage_account}.blob.core.windows.net/{file_path}"

df_btc_tweets = spark.read.option("header", True).option("inferSchema", True).csv(url)

### data source....
 https://www.kaggle.com/datasets/gauravduttakiit/bitcoin-tweets-16m-tweets-with-sentiment-tagged

In [51]:
from pyspark.sql.functions import col, count, isnan, when

df_btc_tweets = spark.read.option("header", True).option("inferSchema", True).csv(url)

print("Schema:")
df_btc_tweets.printSchema()

print("\nNull values per column:")
null_counts = df_btc_tweets.select([
    count(when(col(c).isNull() | isnan(c), c)).alias(c)
    for c in df_btc_tweets.columns
])
null_counts.show(truncate=False)

### Clean the data

In [52]:
column_names = ["date", "text", "sentiment"]

df_btc_tweets = spark.read.option("header", False).csv(url).toDF(*column_names)
df_btc_tweets_cleaned = df_btc_tweets.filter(df_btc_tweets["date"] != "Date")

In [53]:
df_btc_tweets_cleaned = df_btc_tweets.filter(df_btc_tweets["date"] != "Date")
df_btc_tweets_cleaned.show(5, truncate=False)

In [54]:
from pyspark.sql.functions import trim

df_final = df_btc_tweets_cleaned.filter(
    (df_btc_tweets_cleaned["date"].isNotNull()) & (trim(df_btc_tweets_cleaned["date"]) != "") &
    (df_btc_tweets_cleaned["text"].isNotNull()) & (trim(df_btc_tweets_cleaned["text"]) != "") &
    (df_btc_tweets_cleaned["sentiment"].isNotNull()) & (trim(df_btc_tweets_cleaned["sentiment"]) != "")
)
print("Final row count with valid date, text, and sentiment:", df_final.count())

In [55]:
from pyspark.sql.functions import year, to_date

df_final_with_year = df_final.withColumn("year", year(to_date("date")))

In [56]:
from pyspark.sql.functions import year, to_date

df_final_with_year = df_final.withColumn("year", year(to_date("date")))

df_filtered_years = df_final_with_year.filter(
    (col("year").isNotNull()) &
    (col("year") >= 2014) & (col("year") <= 2019)
)

df_filtered_years.select("year").distinct().orderBy("year").show()
df_filtered_years.groupBy("year").count().orderBy("year").show()

print("Row count after year filtering:", df_filtered_years.count())

In [58]:
valid_sentiments = ["Positive", "Negative", "Neutral"]

df_final_cleaned = df_filtered_years.filter(col("sentiment").isin(valid_sentiments))

In [59]:
df_final_cleaned = df_final_cleaned.filter(col("sentiment").isin(["Positive", "Negative"]))
df_final_cleaned.groupBy("year", "sentiment").count().orderBy("year", "sentiment").show()
df_final_cleaned.groupBy("sentiment").count().orderBy("sentiment").show()
print("Row count after removing 'Neutral':", df_final_cleaned.count())

In [65]:
df_final_cleaned.coalesce(1).write.mode("overwrite").parquet(
    "wasbs://bronze@ds562team9datalake.blob.core.windows.net/Crypto_tweets/cleaned_tweet_data/"
)

In [66]:
df_check = spark.read.parquet("wasbs://bronze@ds562team9datalake.blob.core.windows.net/Crypto_tweets/cleaned_tweet_data/")
df_check.count()
